In [ ]:
# calculate optic flow on the dataset using Lukas Kanade method

In [3]:
import numpy as np
from matplotlib import pyplot as plt
import os, cv2
from scipy.ndimage import gaussian_filter
from scipy.interpolate import griddata
from scipy import io

In [4]:
calib_path = '../CameraCalibration/camera_data/'

# this should be a folder containing the image file directories from Zenodo, with each trajectory zip file unzipped
pngs_path = '../publicdataset/images/' 

# these flows are pre-calculated in the flows.zip folder on Zenodo, but this script can be used to recalculate them and save them to outpath
out_path = '../publicdataset/flows/LK' 

In [5]:
os.listdir(calib_path)

['insta2_spherical_coordinates_water_xyz.mat',
 'insta1_pixel_value_LUT.mat',
 'insta1_mtf.mat',
 'insta2_pixel_value_LUT.mat',
 'insta2_mtf.mat',
 'insta2_spectral_sensitivity.mat',
 'insta2_camIntrinsics_water.mat',
 'insta1_spectral_sensitivity.mat',
 'insta1_camIntrinsics_water.mat',
 '.DS_Store',
 'dive_case_proj_clipped.npz',
 'equirectangular_projmaps.npz',
 'insta1_spherical_coordinates_water_xyz.mat',
 'dive_case_image.png',
 'README']

In [6]:
# file management

insta1dates = ['Oct16','Oct17','Oct19']
insta2dates = ['Oct12','Oct13','Oct14','Oct15']

def getinsta(filename):
    insta = -1
    for date in insta1dates:
        if date in filename:
            insta = 1
    for date in insta2dates:
        if date in filename:
            if insta == 1:
                print(f'error in getinsta on {filename}')
            insta = 2
    return insta

def listpngs(vid):
    rawpngs = os.listdir(vid)
    pngnums = sorted([int(png[:-4]) for png in rawpngs])
    pngs = [os.path.join(vid,str(pngnum)+'.png') for pngnum in pngnums]
    return pngs

def getpngs(speed,date,sec):
    viddir = f'{pngs_path}{speed}/{date}'
    if  not  os.path.exists(viddir):
        return []
    vid = os.listdir(viddir)[0]
    pngs = listpngs(os.path.join(viddir,vid))
    if len(pngs) < sec*fps:
        return pngs
    else:
        first = sec*fps
        last = min(len(pngs),(sec+1)*fps+2*timestep+1) #updated from (sec+1)*fps+timestep before cleanup 
        return pngs[first:last]

In [7]:
# 3D projection

pts3d = np.zeros([1504*1504,3,3,2])
for insta in [1,2]:
    for cam in [0,1]:
        #calibdata = io.loadmat(os.path.join(calib_path,f'spherical_coordinates_insta{insta}_water.mat'))
        calibdata = io.loadmat(os.path.join(calib_path,f'insta{insta}_spherical_coordinates_water_xyz.mat'))
        pts3d[:,0,insta,cam] = np.ndarray.flatten(calibdata[f'sphericalCam{cam+1}G']['x'][0][0])
        pts3d[:,1,insta,cam] = np.ndarray.flatten(calibdata[f'sphericalCam{cam+1}G']['y'][0][0])
        pts3d[:,2,insta,cam] = np.ndarray.flatten(calibdata[f'sphericalCam{cam+1}G']['z'][0][0])

def Rx(gamma):
    return [[1,0,0],[0,np.cos(gamma),-np.sin(gamma)],[0,np.sin(gamma),np.cos(gamma)]]

def Ry(beta):
    return [[np.cos(beta),0,np.sin(beta)],[0,1,0],[-np.sin(beta),0,np.cos(beta)]]

def Rz(alpha):
    return [[np.cos(alpha),-np.sin(alpha),0],[np.sin(alpha),np.cos(alpha),0],[0,0,1]]

def makeR(yaw,pitch=0,roll=0):
    return np.matmul(np.matmul(Rx(pitch),Ry(yaw)),Rz(roll))

colorchannel = 1
sig1 = 3
sig2 = 2
case = plt.imread(f'{calib_path}dive_case_image.png')[:,:,1]
case = (case>0.5).astype('uint8')

def tanlumfromfile(filename,pitch=0,yaw=0,FOV=40,res=40,foclen=1.0,colorchannel=colorchannel):
    insta = getinsta(filename)
    im = cv2.imread(filename)[:,:,colorchannel]
    im = im*case
    if np.abs(yaw) < pi/2:
        cam = 0
        im = gaussian_filter(np.rot90(im[:,:1504],3),sig1)
    else:
        cam = 1
        yaw = (yaw-pi)%pi
        im = gaussian_filter(np.rot90(im[:,1504:]),sig2)
    R = makeR(yaw,pitch)
    rotpts3d = np.matmul(pts3d[:,:,insta,cam],R)
    X = rotpts3d[:,0].reshape([1504,1504])
    Y = rotpts3d[:,1].reshape([1504,1504])
    Z = rotpts3d[:,2].reshape([1504,1504])
    x = X*foclen/(Z+1e-10)
    y = Y*foclen/(Z+1e-10)
    lim = np.tan(FOV/2*pi/180)
    clip = ~np.logical_or(np.logical_or(x>lim,x<-lim),np.logical_or(y>lim,y<-lim))
    pixx,pixy = np.meshgrid(np.linspace(-lim,lim,res),np.linspace(-lim,lim,res))
    interp = griddata((x[clip],y[clip]),im[clip],(pixx,pixy),'cubic')
    return interp

def pts2xy(p0,remnans=False,FOV=40,pixperdeg=1):
    x0y0 = np.reshape(p0[:50],[5,5,2])
    x0 = np.ndarray.flatten(x0y0[:,:,0])
    y0 = np.ndarray.flatten(x0y0[:,:,1])
    sz = FOV*pixperdeg
    scale = np.tan(FOV/2*pi/180)/(sz/2)
    x0 = scale*(x0-sz/2)
    y0 = scale*(y0-sz/2)
    if remnans:
        x = np.array([x0[i] for i in range(len(x0)) if not (np.isnan(x0[i]) or np.isnan(y0[i]))]) 
        y = np.array([y0[i] for i in range(len(y0)) if not (np.isnan(x0[i]) or np.isnan(y0[i]))])
    else:
        x = x0
        y = y0
    return x,y

In [6]:
# optic flow
timestep = 5
fps = 100
RF = 40
timestep = 5
levels = 2
    
patchsz = 25
pi = np.pi
pitches = np.multiply(pi/180,np.linspace(40,-60,11))

def procLK(speed,date,sec,pngs=None,initi=-1,numframes=100,pixperdeg=1):
    
    # optic flow params that change with resolution
    wsize = int(pixperdeg*RF/2**levels)
    feature_params = dict(maxCorners = 25, 
                          qualityLevel = 1e-9,
                          minDistance = int(pixperdeg*RF/6),
                          blockSize = int(pixperdeg*RF/6))
    lk_params = dict(winSize = (wsize,wsize),
                     maxLevel = levels,
                     criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10, 0.03)) 
    
    # set up image list
    isbackward = False
    if 'Oct13' in date and 'R+50' in speed:
        isbackward = True
        speed = 'R-50'
    if ('Oct13' in date or 'Oct14' in date) and ('T' in speed):
        isbackward = True

    if pngs is None:
        pngs = getpngs(speed,date,sec)
    if len(pngs) == 0:
        print(f'0 pngs in {speed_date_sec}')
        return
    if isbackward:
        pngs.reverse()

    outfile = f'{out_path}{speed_date_sec}_{pixperdeg}'

    # start or restart
    if initi == -1:
        p0s = np.nan*np.ones([len(pitches),numframes,patchsz*2*2]) # 2 image patches, 2D samples
        p1s = np.nan*np.ones_like(p0s)
        matcherrs = np.nan*np.ones([len(pitches),numframes,patchsz*2])
        matchsts = np.nan*np.ones_like(matcherrs)
    else:
        inprogress = np.load(f'{outfile}.npz',allow_pickle=True)
        p0s = inprogress['p0s']
        p1s = inprogress['p1s']
        matcherrs = inprogress['matcherrs']
        matchsts = inprogress['matchsts']

    i = initi
    while True:
        if i%10 == 0:
            np.savez(outfile,p0s=p0s,p1s=p1s,matcherrs=matcherrs,matchsts=matchsts)

        # skip if there's a keyframe up next
        framenums = [int(pngs[j].split(os.sep)[-1][:-4]) for j in range(i,i+timestep+1)]
        iskeyframe = np.any([framenum%96 == 0 for framenum in framenums]) 
        if iskeyframe:
            i = i+1+timestep
        else:
            i = i+1
        
        if i+1+timestep>=len(pngs) or i>=numframes:
            break
        
        for p in range(len(pitches)):

            im1_cam1 = tanlumfromfile(pngs[i],pitch=pitches[p],yaw=0,FOV=RF,res=pixperdeg*RF+2)[1:-1,1:-1]
            im2_cam1 = tanlumfromfile(pngs[i+timestep],pitch=pitches[p],yaw=0,FOV=RF,res=pixperdeg*RF+2)[1:-1,1:-1]

            p0 = cv2.goodFeaturesToTrack(im1_cam1.astype('uint8'),mask=None,**feature_params)
            if p0 is None:
                continue
            p1, st, err = cv2.calcOpticalFlowPyrLK(im1_cam1.astype('uint8'),im2_cam1.astype('uint8'),p0,None,**lk_params)
            npts = len(p0)

            # record flow output
            p0s[p,i,:2*npts] = np.ndarray.flatten(p0)
            p1s[p,i,:2*npts] = np.ndarray.flatten(p1) # get rid of bad ones though? -> no, keep all for now.
            matcherrs[p,i,:npts] = np.squeeze(err)
            matchsts[p,i,:npts] = np.squeeze(st)

            # CAMERA 2:
            # get ims from cam2
            im1_cam2 = tanlumfromfile(pngs[i],pitch=pitches[p],yaw=pi,FOV=RF,res=pixperdeg*RF+2)[1:-1,1:-1]
            im2_cam2 = tanlumfromfile(pngs[i+timestep],pitch=pitches[p],yaw=pi,FOV=RF,res=pixperdeg*RF+2)[1:-1,1:-1]

            # get pts and flow from cam2
            p0 = cv2.goodFeaturesToTrack(im1_cam2.astype('uint8'),mask=None,**feature_params)
            if p0 is None:
                continue

            p1, st, err = cv2.calcOpticalFlowPyrLK(im1_cam2.astype('uint8'),im2_cam2.astype('uint8'),p0,None,**lk_params)
            npts = len(p0)
            # record flow output
            p0s[p,i,2*patchsz:2*patchsz+2*npts] = np.ndarray.flatten(p0)
            p1s[p,i,2*patchsz:2*patchsz+2*npts] = np.ndarray.flatten(p1) # get rid of bad ones though? -> no, keep all for now.
            matcherrs[p,i,patchsz:patchsz+npts] = np.squeeze(err)
            matchsts[p,i,patchsz:patchsz+npts] = np.squeeze(st)
    np.savez(outfile+'_final',p0s=p0s,p1s=p1s,matcherrs=matcherrs,matchsts=matchsts)
    return p0s,p1s,matcherrs,matchsts

def cleanupLK(speed,date,sec):
    pngs = getpngs(speed,date,sec)
    fname = f'{out_path}{speed_date_sec}_final.npz'
    if not os.path.exists(fname):
        return
    preexisting = np.loadz(fname,allow_pickle=True)
    initi = 100-np.sum([np.all(np.isnan(preexisting['matchsts'][:,i,:])) for i in range(100)])
    procLK(speed,date,sec,pngs=pngs,initi=initi)
    return

def start_or_restart(speed,date,sec,pixperdeg=1):
    finalfile = f'{out_path}{speed_date_sec}_{pixperdeg}_final.npz'
    if os.path.exists(finalfile):
        print('\t\tcomplete')
        return
    
    inprogressfile = f'{out_path}{speed_date_sec}_{pixperdeg}.npz'
    if not os.path.exists(inprogressfile):
        print('\tstarting from scratch')
        procLK(speed,date,sec)
    else:
        speed,date,sec = speed_date_sec.split('_')
        pngs = getpngs(speed,date,int(sec))
        preexisting = np.load(inprogressfile,allow_pickle=True)
        initi = 100-np.sum([np.all(np.isnan(preexisting['matchsts'][:,i,:])) for i in range(100)])
        if initi != 0:
            print(f'\tfinishing last {100-initi} frames')
            procLK(speed,date,sec,pngs=pngs,initi=initi)
    return

In [7]:
dates = [f'Oct1{i}' for i in range(2,10) if i!=8]
dates.remove('Oct18')
speeds = os.listdir(pngs_path)

In [8]:
for speed in speeds:
    print(speed,flush=True)
    for date in dates:
        print('\t'+date,flush=True)
        for sec in range(4):
            print('\t\t'+str(sec),flush=True)
            start_or_restart(speed,date,sec,pixperdeg=1)

T40XY
	Oct12
		0
	complete
		1
	complete
		2
	complete
		3
	complete
	Oct13
		0
	complete
		1
	complete
		2
	complete
		3
	complete
	Oct14
		0
	complete
		1
	complete
		2
	complete
		3
	complete
	Oct15
		0
	complete
		1
	complete
		2
	complete
		3
	complete
	Oct16
		0
	complete
		1
	complete
		2
	complete
		3
	complete
	Oct17
		0
	complete
		1
	complete
		2
	complete
		3
	complete
	Oct19
		0
	complete
		1
	complete
		2
	complete
		3
	finishing last 44 frames
